In [ ]:
import requests
import numpy as np
from lxml import html
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client['lenta_news']
collection = db.fresh_news

header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36'}
url = 'https://lenta.ru/'
response = requests.get(url, headers=header)
dom = html.fromstring(response.text)
# items = dom.xpath("//a[contains(@class, 'card-mini')] | //a[contains(@class, 'card-big')]")
items = dom.xpath("//a[contains(@class, 'card-mini')]")
count = 0
for item in items:
    news = {}

    news['title'] = item.xpath(".//span[contains(@class, 'title')]/text()")[0]

    href = item.xpath('.//@href')[0]
    if href[0:4] != 'http':
        news['href'] = url[:-1] + href
        news['source'] = 'LENTA.RU'
    else:
        news['href'] = href
        news['source'] = 'external source'


    try:
        if news['source'] == 'LENTA.RU':
            # news['date'] = '.'.join(href.split('/')[2:5]) + ' ' + item.xpath(".//time[contains(@class, '__date')]/text()")[0]
            news['date'] = html.fromstring(
                requests.get(news['href'], headers=header).text
            ).xpath("//time[contains(@class, 'topic-header__time')]/text()")[0]
        else:
            try:
                news['date'] = item.xpath(".//time[contains(@class, '__date')]/text()")[0]
            except:
                news['date'] = np.nan

    except:
        news['date'] = np.nan

    collection.insert_one(news)